In [1]:
import pickle
import os
import re
import copy
from collections import Counter

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

import seaborn as sns

In [2]:
%matplotlib inline

In [3]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.optim

In [4]:
from sklearn.metrics import mean_squared_error

In [5]:
if torch.cuda.is_available():
    DEVICE = 'cuda:0'
else:
    DEVICE = 'cpu'

# 1.读取数据

In [6]:
data_df = pd.read_pickle('../data/processed/data_df.pkl')

In [7]:
total_usr = len(set(data_df.usr_id.tolist()))
total_item = len(set(data_df.item_id.tolist()))

# 2.深度模型

In [8]:
class RecModel(torch.nn.Module):
    def __init__(self,
                 usr_num:int,
                 item_num:int,
                 emb_usr_size:int,
                 emb_item_size:int,
                 hidden_size:int):
        '''
        usr_num 和 item_num 为对应用户和物品的总数
        emb_size设定用户和物品的隐变量维度
        hidden_size为交互时的维度
        '''
        super(RecModel,self).__init__()
        self.usr_num = usr_num
        self.item_num = item_num
        self.emb_usr_size = emb_usr_size
        self.emb_item_size = emb_item_size
        self.hidden_size = hidden_size
        
        # 查看显卡设备是否可用 
        if torch.cuda.is_available():
            self.device = 'cuda:0'
        else:
            self.device = 'cpu'
            
        self.UserEmbeddingLayer = torch.nn.Embedding(num_embeddings=self.usr_num,
                                                     embedding_dim=self.emb_usr_size)
        self.ItemEmbeddingLayer = torch.nn.Embedding(num_embeddings=self.item_num,
                                                     embedding_dim=self.emb_item_size)
        
        self.UserFeatureLayer = torch.nn.Linear(in_features=self.emb_usr_size,
                                                out_features=self.hidden_size)
        self.ItemFeatureLayer = torch.nn.Linear(in_features=self.emb_item_size,
                                               out_features=self.hidden_size)
        
        # to device
        self.UserEmbeddingLayer.to(self.device)
        self.UserFeatureLayer.to(self.device)
        self.ItemEmbeddingLayer.to(self.device)
        self.ItemFeatureLayer.to(self.device)
        
        
        
    def forward(self,uid_batch,iid_batch):
        '''输入一个batch的usr和item进行交互
        '''
        u_batch_tensor = uid_batch
        i_batch_tensor = iid_batch

        # 装入设备
        u_batch_tensor = u_batch_tensor.to(self.device)
        i_batch_tensor = i_batch_tensor.to(self.device)
        
        # 嵌入 向量化
        
        u_emb_tensor = self.UserEmbeddingLayer(u_batch_tensor)
        i_emb_tensor = self.ItemEmbeddingLayer(i_batch_tensor)
        
        # 特征抽取 和 非线性化
        u_feature = self.UserFeatureLayer(u_emb_tensor)
        i_feature = self.ItemFeatureLayer(i_emb_tensor)
        
        u_feature = torch.relu(u_feature)
        i_feature = torch.relu(i_feature)
        
        batch_size = u_feature.shape[0]
        u_feature = u_feature.reshape(batch_size,1,self.hidden_size)
        i_feature = i_feature.reshape(batch_size,self.hidden_size,1)
        
        output = torch.bmm(u_feature,i_feature)
        output = torch.squeeze(output)

        return output

In [9]:
# nn = RecModel(usr_num=total_usr,item_num=total_item,emb_usr_size=50,emb_item_size=150,hidden_size=25)

# sample = data_df.sample(n=32)

# usr_batch = [int(i) for i in sample.usr_id.tolist()]
# item_batch = [int(i) for i in sample.item_id.tolist()]


# usr_batch = torch.tensor(usr_batch)
# item_batch = torch.tensor(item_batch)
# nn(usr_batch,item_batch)

### 交叉验证

In [10]:
class MLBaseDataSet(Dataset):
    def __init__(self,df):
        '''
        输入df 构造dataset
        输出 样本编号和标注
        '''
        self.df = df.copy()
        # 注意 原始数据用户id和物品id从1开始的，但是在embedding过程中是从0算的，因此此处减一
        self.df['usr_id'] = df['usr_id'].apply(lambda x:int(x)-1)
        self.df['item_id'] = df['item_id'].apply(lambda x:int(x)-1)
        self.df['rating'] = df['rating'].apply(lambda x:int(x))
        
    def __len__(self):
        return self.df.shape[0]
    def __getitem__(self,idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample_df = self.df.iloc[idx]
        sample = (sample_df.usr_id,sample_df.item_id,sample_df.rating)
        return sample

In [11]:
cv_df_lst = []
for i in range(5):
    df_fname = '../data/processed/cv_{0}_df.pkl'.format(i+1)
    df = pd.read_pickle(df_fname)
    cv_df_lst.append(df)

### 定义模型、优化器、损失函数

In [12]:
model = RecModel(usr_num=total_usr,item_num=total_item,emb_usr_size=50,emb_item_size=150,hidden_size=25)

In [13]:
model

RecModel(
  (UserEmbeddingLayer): Embedding(943, 50)
  (ItemEmbeddingLayer): Embedding(1682, 150)
  (UserFeatureLayer): Linear(in_features=50, out_features=25, bias=True)
  (ItemFeatureLayer): Linear(in_features=150, out_features=25, bias=True)
)

In [14]:
optimizer = torch.optim.SGD([
        {'params': model.parameters()},
                ], lr=0.005,momentum=0.9)

In [15]:
BATCH_SIZE = 1024

### 训练测试过程

仅训练一个step的结果

In [16]:
mse_lst = []

for test_idx in range(5):
    print('----------------')
    print('Test on cv_{0}_df'.format(test_idx+1))
    train_idx_lst = [i for i in range(5) if i!=test_idx]
    # 训练过程
    train_df_lst = []
    for train_idx in train_idx_lst:
        df = cv_df_lst[train_idx]
        train_df_lst.append(df[df['type']=='test'])
    train_df = pd.concat(train_df_lst)
    train_dataset = MLBaseDataSet(train_df)
    train_dataloader = DataLoader(train_dataset,batch_size=BATCH_SIZE)
    

    model.train()
    
    print('Starting training porcess...')
    for num,(uid_batch,iid_batch,true_y) in enumerate(train_dataloader):
        optimizer.zero_grad()
        criterion = torch.nn.MSELoss()
        
        uid_batch = torch.tensor(uid_batch)
        iid_batch = torch.tensor(iid_batch)
        
        pred_y = model(uid_batch,iid_batch)
        true_y = true_y.to(torch.float).to(DEVICE)
        loss = criterion(pred_y,true_y)
        loss.requires_grad_(True)
        loss.backward()
        print('Batch:{0} Loss:{1}'.format(num,loss))
        optimizer.step()
    
    # 测试过程
    model.eval()
    df = cv_df_lst[test_idx]
    test_df = df[df['type']=='test']

    test_dataset = MLBaseDataSet(test_df)
    test_dataloader = DataLoader(test_dataset,batch_size=BATCH_SIZE)
    
    temp_mse_lst = []
    # 分batch进行MSE计算 最后平均
    for num,(test_uid_batch,test_iid_batch,test_true_y) in enumerate(train_dataloader):
        
        test_uid_batch = torch.tensor(test_uid_batch)
        test_iid_batch = torch.tensor(test_iid_batch)
        
        test_pred_y = model(test_uid_batch,test_iid_batch)
        test_pred_y = test_pred_y.cpu().detach().numpy()
        
        mse_batch = mean_squared_error(test_true_y,test_pred_y)
        temp_mse_lst.append(mse_batch)
        
    mse = np.average(temp_mse_lst)
    mse_lst.append(mse)
    print('MSE on cv_{0}_df:{1}'.format(test_idx+1,mse))
print('Average MSE:{0}'.format(np.average(mse_lst)))

----------------
Test on cv_1_df
Starting training porcess...


<ipython-input-16-f4252c8ac7d3>:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  uid_batch = torch.tensor(uid_batch)
<ipython-input-16-f4252c8ac7d3>:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  iid_batch = torch.tensor(iid_batch)


Batch:0 Loss:6.874739646911621
Batch:1 Loss:7.875307083129883
Batch:2 Loss:7.199061870574951
Batch:3 Loss:8.183090209960938
Batch:4 Loss:5.710568428039551
Batch:5 Loss:5.983312606811523
Batch:6 Loss:5.4632391929626465
Batch:7 Loss:5.467398643493652
Batch:8 Loss:5.4084906578063965
Batch:9 Loss:4.5362396240234375
Batch:10 Loss:3.292048215866089
Batch:11 Loss:3.2019824981689453
Batch:12 Loss:3.4166274070739746
Batch:13 Loss:3.1472904682159424
Batch:14 Loss:2.788240432739258
Batch:15 Loss:3.528219223022461
Batch:16 Loss:3.5508158206939697
Batch:17 Loss:3.251126289367676
Batch:18 Loss:3.4667694568634033
Batch:19 Loss:3.880143880844116
Batch:20 Loss:3.2743868827819824
Batch:21 Loss:3.716200113296509
Batch:22 Loss:3.4737396240234375
Batch:23 Loss:3.05910587310791
Batch:24 Loss:2.7399075031280518
Batch:25 Loss:2.456937313079834
Batch:26 Loss:2.331902503967285
Batch:27 Loss:2.411445379257202
Batch:28 Loss:2.5245649814605713
Batch:29 Loss:2.3266654014587402
Batch:30 Loss:2.6535511016845703
Batch

<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

...

<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

MSE on cv_5_df:1.0687302366492213
Average MSE:1.2143274678265161


<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_iid_batch = torch.tensor(test_iid_batch)
<ipython-input-16-f4252c8ac7d3>:47: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  test_uid_batch = torch.tensor(test_uid_batch)
<ipython-input-16-f4252c8ac7d3>:48: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone()

### 对比lgbRegression

```
----------------
Test on cv_1_df
MSE on cv_5_df:0.9916941295553173
----------------
Test on cv_2_df
MSE on cv_5_df:0.9696588575493511
----------------
Test on cv_3_df
MSE on cv_5_df:0.9576872585148317
----------------
Test on cv_4_df
MSE on cv_5_df:0.9672447937961466
----------------
Test on cv_5_df
MSE on cv_5_df:0.97248790766502
Average MSE:0.9717545894161332
```

# 3.模型导出和权重保存

### 权重保存

In [18]:
torch.save(model.state_dict,'./RecModelWeights.bin')

### 脚本化保存

In [20]:
scripted_model = torch.jit.script(model)

In [21]:
scripted_model.save('./RecModel.pt')

In [31]:
loaded_model = torch.jit.load('./RecModel.pt')

In [32]:
loaded_model

RecursiveScriptModule(
  original_name=RecModel
  (UserEmbeddingLayer): RecursiveScriptModule(original_name=Embedding)
  (ItemEmbeddingLayer): RecursiveScriptModule(original_name=Embedding)
  (UserFeatureLayer): RecursiveScriptModule(original_name=Linear)
  (ItemFeatureLayer): RecursiveScriptModule(original_name=Linear)
)

In [47]:
for i in model.state_dict().keys():
    print(i)

UserEmbeddingLayer.weight
ItemEmbeddingLayer.weight
UserFeatureLayer.weight
UserFeatureLayer.bias
ItemFeatureLayer.weight
ItemFeatureLayer.bias


In [39]:
loaded_model.state_dict()['UserEmbeddingLayer.weight']

tensor([[ 0.0043, -0.8500,  0.0737,  ...,  1.2802, -0.1540,  0.1368],
        [ 0.1400, -0.4781, -1.3062,  ..., -0.2554, -0.2076, -0.8289],
        [-1.4320,  0.4104,  1.6189,  ...,  1.3352,  2.4788,  0.3984],
        ...,
        [ 0.7638,  0.0089,  1.0013,  ...,  0.2047, -1.3364,  0.3445],
        [ 0.0680,  2.7619, -0.1530,  ...,  0.4327,  0.0616, -1.3079],
        [ 0.6047, -0.2316,  0.2670,  ..., -0.2058, -0.2006,  1.1614]])

In [40]:
model.state_dict()['UserEmbeddingLayer.weight']

tensor([[ 0.0043, -0.8500,  0.0737,  ...,  1.2802, -0.1540,  0.1368],
        [ 0.1400, -0.4781, -1.3062,  ..., -0.2554, -0.2076, -0.8289],
        [-1.4320,  0.4104,  1.6189,  ...,  1.3352,  2.4788,  0.3984],
        ...,
        [ 0.7638,  0.0089,  1.0013,  ...,  0.2047, -1.3364,  0.3445],
        [ 0.0680,  2.7619, -0.1530,  ...,  0.4327,  0.0616, -1.3079],
        [ 0.6047, -0.2316,  0.2670,  ..., -0.2058, -0.2006,  1.1614]])

In [ ]:
torch.jit.trace_module()

# 4.测试ONNX

In [21]:
sample = data_df.sample(n=32)

usr_batch = [int(i) for i in sample.usr_id.tolist()]
item_batch = [int(i) for i in sample.item_id.tolist()]


usr_batch = torch.tensor(usr_batch)
item_batch = torch.tensor(item_batch)

In [22]:
dummpy_input = (usr_batch,item_batch)

In [24]:
torch.onnx.export(model=onnx_model,args=dummpy_input,f='./Rec_ONNX_model.onx')